<img src="imagenes/rn3.png" width="200">
<img src="http://www.identidadbuho.uson.mx/assets/letragrama-rgb-150.jpg" width="200">

# [Curso de Redes Neuronales](https://curso-redes-neuronales-unison.github.io/Temario/)

# Una red neuronal multicapa simple usando TensorFlow


[**Julio Waissman Vilanova**](http://mat.uson.mx/~juliowaissman/), 27 de septiembre de 2017.



En esta libreta se muestra el ejemplo básico para una red multicapa sencilla
aplicada al conjunto de datos [MNIST](http://yann.lecun.com/exdb/mnist/).

Esta libreta es básicamente una traducción del ejemplo
desarrollado por [Aymeric Damien](https://github.com/aymericdamien/TensorFlow-Examples/)


In [1]:
import tensorflow as tf

## 1. Cargar datos

Primero cargamos los archivos que se utilizan para el aprendizaje. Para otro tipo de problemas, es necesario hacer un proceso conocido como *Data Wrangling*, que normalmente se realiza con la ayuda de *Pandas*. 

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from t

Para que un aprendizaje tenga sentido es necesario tener bien separado un conjunto de datos de aprendizaje y otro de prueba (en caso de grandes conjuntos de datos es la opción). Como vemos tanto las imágenes como las etiquetas están separados en archivos de datos y de aprendizaje.

El objeto `mnist` es un objeto tensorflow que contiene 3 objetos tipo tensorflow: *test*, *train* y *validation*, los cuales a su vez contienen *ndarrays* de *numpy*. La estructura es la misma para cada conjunto de datos. Veamos su estructura:


In [3]:
print("Tipo de images: {}".format(type(mnist.train.images)))
print("Tipo de epochs_completed: {}".format(type(mnist.train.epochs_completed)))
print("Tipo de labels: {}".format(type(mnist.train.labels)))
print("Tipo de nest_batch: {}".format(type(mnist.train.next_batch)))
print("Tipo de num_examples: {}".format(type(mnist.train.num_examples)))

Tipo de images: <class 'numpy.ndarray'>
Tipo de epochs_completed: <class 'int'>
Tipo de labels: <class 'numpy.ndarray'>
Tipo de nest_batch: <class 'method'>
Tipo de num_examples: <class 'int'>


Como generar el conjunto de datos para ser utilizado dentro de TensorFlow es objeto de otra libreta. Por el momento concentremonos en como hacer una red neuronal rápido y sin dolor.

Sin embargo, vamos a ver unos cuantos datos que nos pueden ser de útilidad para la construcción de la red neuronal.

In [4]:
print("Forma del ndarray con las imágenes: {}".format(mnist.train.images.shape))
print("Forma del ndarray con las etiquetas: {}".format(mnist.train.labels.shape))
print("-" * 79)
print("Número de imagenes de entrenamiento: {}".format(mnist.train.images.shape[0]))
print("Tamaño de las imagenes: {}".format(mnist.train.images.shape[1]))
print("Clases diferentes: {}".format(mnist.train.labels.shape[1]))


Forma del ndarray con las imágenes: (55000, 784)
Forma del ndarray con las etiquetas: (55000, 10)
-------------------------------------------------------------------------------
Número de imagenes de entrenamiento: 55000
Tamaño de las imagenes: 784
Clases diferentes: 10


## 2. Construcción de la red neuronal

Para hacer una red neuronal lo más genérica posible y que pdamos reutilizar en otros proyectos, vamos a establecer los parámetros base independientemente de la inicialización de la red, independientemente de la forma en que construimos la red. 

Comencemos por establecer una función genérica que nos forme una red neuronal con dos capas ocultas. No agrego más comentarios porque, con la experiencia de las libretas anteriores, la construcción de la red neuronal se explica sola.

In [5]:
def red_neuronal_dos_capas_ocultas(x, pesos, sesgos):
    """
    Genera una red neuronal de dos capas para usar en TensorFlow
    
    Parámetros
    ----------
    pesos: un diccionario con tres etiquetas: 'h1', 'h2' y 'ho'
           en donde cada una es una tf.Variable conteniendo una 
           matriz de dimensión [num_neuronas_capa_anterior, num_neuronas_capa]
                  
    sesgos: un diccionario con tres etiquetas: 'b1', 'b2' y 'bo'
            en donde cada una es una tf.Variable conteniendo un
            vector de dimensión [numero_de_neuronas_capa]
                   
    Devuelve
    --------
    Un ops de tensorflow que calcula la salida de una red neuronal
    con dos capas ocultas, y activaciones RELU.
    
    """
    # Primera capa oculta con activación ReLU
    capa_1 = tf.matmul(x, pesos['h1'])
    capa_1 = tf.add(capa_1, sesgos['b1'])
    capa_1 = tf.nn.relu(capa_1)
    
    # Segunda capa oculta con activación ReLU
    capa_2 = tf.matmul(capa_1, pesos['h2'])
    capa_2 = tf.add(capa_2, sesgos['b2'])
    capa_2 = tf.nn.relu(capa_2)
    
    # Capa de salida con activación lineal
    # En Tensorflow la salida es siempre lineal, y luego se especifica
    # la función de salida a la hora de calcularla como vamos a ver 
    # más adelante
    capa_salida = tf.matmul(capa_2, pesos['ho']) + sesgos['bo']
    return capa_salida

Y ahora necesitamos poder generar los datos de entrada a la red neuronal de
alguna manera posible. Afortunadamente sabemos exactamente que necesitaos, así
que vamos a hacer una función que nos genere las variables de peso y sesgo.

Por el momento, y muy a la brava, solo vamos a generarlas con números aletorios con una 
distribución $\mathcal{N}(0, 1)$.

In [6]:
def inicializa_pesos(entradas, n1, n2, salidas):
    """
    Genera un diccionario con pesos  
    para ser utilizado en la función red_neuronal_dos_capas_ocultas
    
    Parámetros
    ----------
    entradas: Número de neuronas en la capa de entrada
    
    n1: Número de neuronas en la primer capa oculta
    
    n2: Número de neuronas en la segunda capa oculta
    
    salidas: Número de neuronas de salida
    
    Devuelve
    --------
    Dos diccionarios, uno con los pesos por capa y otro con los sesgos por capa
    
    """
    pesos = {
        'h1': tf.Variable(tf.random_normal([entradas, n1])),
        'h2': tf.Variable(tf.random_normal([n1, n2])),
        'ho': tf.Variable(tf.random_normal([n2, salidas]))
    }
    
    sesgos = {
        'b1': tf.Variable(tf.random_normal([n1])),
        'b2': tf.Variable(tf.random_normal([n2])),
        'bo': tf.Variable(tf.random_normal([salidas]))
    }
    
    return pesos, sesgos

Ahora necesitamos establecer los parámetros de la topología de la red neuronal. 
Tomemos en cuenta que estos prámetros los podríamos haber establecido desde
la primer celda, si el fin es estar variando los parámetros para escoger los que 
ofrezcan mejor desempeño.

In [7]:
num_entradas = 784  #  Lo sabemos por la inspección que hicimos a mnist
num_salidas = 10    # Ídem

# Aqui es donde podemos jugar
num_neuronas_capa_1 = 256
num_neuronas_capa_2 = 256

¡A construir la red! Para esto vamos a necesitar crear las entradas
con un placeholder, y crear nuestra topología de red neuronal.

Observa que la dimensión de x será [None, num_entradas], lo que significa que 
la cantidad de renglones es desconocida (o variable).

In [8]:
# La entrada a la red neuronal
x = tf.placeholder("float", [None, num_entradas])

# Los pesos y los sesgos
w, b = inicializa_pesos(num_entradas, num_neuronas_capa_1, num_neuronas_capa_2, num_salidas)

# Crea la red neuronal
estimado = red_neuronal_dos_capas_ocultas(x, w, b)


Parecería que ya está todo listo. Sin ambargo falta algo muy importante: No hemos explicado
ni cual es el criterio de error (loss) que vamos a utilizar, ni cual va a ser el método de
optimización (aprendizaje) que hemos decidido aplicar.

Primero definamos el costo que queremos minimizar, y ese costo va a estar en función de lo
estimado con lo real, por lo que necesitamos otra entrada de datos para los datos de salida.

Sin ningun lugar a dudas, el costo que mejor describe este problema es el de *softmax*


In [9]:
#  Creamos la variable de datos de salida conocidos
y = tf.placeholder("float", [None, num_salidas])

#  Definimos la función de costo
costo = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=estimado, labels=y))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



Y ahora definimos que función de aprendizaje vamos a utilizar. Existen muchas funciones
de aprendizaje en tensorflow, las cuales se pueden consultar en `tf.train.`. Entre las
existentes podemos ver algunas conocidas del curso como descenso de gradiente simple,
momento, rprop, rmsprop entre otras. Casi todas las funciones de optimización (aprendizaje)
acaban su nombre con `Optimize`.

En este caso vamos a usar un método comocido como el *algoritmo de Adam* el cual 
se puede consultar [aqui](http://arxiv.org/pdf/1412.6980.pdf). El metodo utiliza dos calculos
de momentos diferentes, y por lo visto genera resultados muy interesantes desde el punto 
de vista práctico.

¿Cual es el mejor método? Pues esto es en función de tu problema y de la cantidad de datos que tengas.
Lo mejor es practicar con varios métodos para entender sus ventajas y desventajas.

En todo caso el método de optimización requiere que se le inicialice con una tasa de aprendizaje.

In [10]:
alfa = 0.001
optimizador = tf.train.AdamOptimizer(learning_rate=alfa)
paso_entrenamiento = optimizador.minimize(costo)


## 3. Ejecutar la sesión usando mini-batches

Ahora, ya que la red neuronal está lista vamos a ejecutar la red utilizando el algoritmo de
Adam pero en forma de mini-batches. Con el fin de tener control sobre el problema, vamos a establecer un número máximo de epoch (ciclos de aprendizaje), el tamaño de los mini-batches, y cada cuandos epoch 
quisieramos ver como está evolucionando la red neuronal.

Como entrenar una red neuronal no tiene sentido, si no es porque la queremos usar para reconocer,
no tendría sentido entrenarla y luego perderla y tener que reentrenar en cada ocasión. Recuerda que cuando
se cierra la sesión se borra todo lo que se tenía en memoria. 

Para esto vamos a usar una ops especial llamada `Saver`, que permite guardar en un archivo la red neuronal y 
después utilizarla en otra sesión (en otro script, computadora, ....).


In [13]:
archivo_modelo = "/tmp/rnn2.ckpt"
saver = tf.train.Saver()

Como todo se ejecuta dentro de una sesión, no es posible hacerlo por partes (si usamos el 
`with` que debería ser la única forma en la que iniciaramos una sesión). Por lo tanto procuraré dejar comentado el código.

In [14]:
numero_epochs = 30
tamano_minibatch = 100
display_step = 1

# Muy importante la primera vez que se ejecuta inicializar todas las variables
init = tf.global_variables_initializer()

# La manera correcta de iniciar una sesión y realizar calculos
with tf.Session() as sess:
    sess.run(init)

    # Ciclos de entrenamiento
    for epoch in range(numero_epochs):

        #  Inicializa el costo promedio de todos los minibatches en 0
        avg_cost = 0.
        
        #  Calcula el número de minibatches que se pueden usar 
        total_batch = int(mnist.train.num_examples/tamano_minibatch)

        #  Por cada minibatch
        for i in range(total_batch):
            
            #  Utiliza un generador incluido en mnist que obtiene 
            #  tamano_minibatch ejemplos selecionados aleatoriamente del total
            batch_x, batch_y = mnist.train.next_batch(tamano_minibatch)
            
            #  Ejecuta la ops del paso_entrenamiento para aprender 
            #  y la del costo, con el fin de mostrar el aprendizaje
            _, c = sess.run([paso_entrenamiento, costo], feed_dict={x: batch_x, y: batch_y})
            
            #  Calcula el costo del minibatch y lo agrega al costo total
            avg_cost += c / total_batch
        
        # Muestra los resultados
        if epoch % display_step == 0:
            print (("Epoch: " + str(epoch)).ljust(20)
                   + ("Costo: " + str(avg_cost)))
    
    #  Guarda la sesión en el archivo rnn2.cptk
    saver.save(sess, archivo_modelo)
    
    print("Se acabaron los epochs, saliendo de la sesión de tensorflow.")

Epoch: 0            Costo: 170.505529338
Epoch: 1            Costo: 41.1230651231
Epoch: 2            Costo: 26.3631318883
Epoch: 3            Costo: 18.5785785967
Epoch: 4            Costo: 13.5170181854
Epoch: 5            Costo: 10.0774533077
Epoch: 6            Costo: 7.6513156884
Epoch: 7            Costo: 5.70684334963
Epoch: 8            Costo: 4.29248864766
Epoch: 9            Costo: 3.31419317846
Epoch: 10           Costo: 2.48717678734
Epoch: 11           Costo: 1.92297998922
Epoch: 12           Costo: 1.49734948321
Epoch: 13           Costo: 1.08213999227
Epoch: 14           Costo: 0.988551660577
Epoch: 15           Costo: 0.846051261536
Epoch: 16           Costo: 0.67428971285
Epoch: 17           Costo: 0.547507881448
Epoch: 18           Costo: 0.595237503391
Epoch: 19           Costo: 0.370802718382
Epoch: 20           Costo: 0.388473673061
Epoch: 21           Costo: 0.394381909823
Epoch: 22           Costo: 0.326050012859
Epoch: 23           Costo: 0.324710129028
Epoch: 2

Ahora vamos a revisar que tan bien realizó el aprendizaje cuando se aplica la red adatos que
no se usaron para entrenamiento. Para esto vamos a utilizar dos ops extas: una 
para definir la operaración de datos bien estimados o mal estimados, y otra para
calcular el promedio de datos bien estimados. Para calcular los datos bien estimados vamos a utilizar `tf.cast` que permite ajustar los tipos
al tipo tensor. 

In [15]:
prediction_correcta = tf.equal(tf.argmax(estimado, 1), tf.argmax(y, 1))

precision = tf.reduce_mean(tf.cast(prediction_correcta, "float"))



Ahora si, vamos a abrir una nueva sesión, vamos a restaurar los valores de la sesión anterior,
y vamos a ejecutar el grafo con el fin de evaluar la ops precision, pero ahora con el
diccionario de alimentación con los datos de prueba.

In [16]:
with tf.Session() as sess:
    sess.run(init)
    saver.restore(sess, archivo_modelo)
    porcentaje_acierto = sess.run(precision, feed_dict={x: mnist.test.images,
                                                        y: mnist.test.labels})
    print("Precisión: {}".format(porcentaje_acierto))


INFO:tensorflow:Restoring parameters from /tmp/rnn2.ckpt
Precisión: 0.9520000219345093


## 4. Contesta las siguientes preguntas

1. ¿Que pasa si aumenta el número de epochs? ¿Cuando deja de ser util aumentar los epoch?

2. ¿Que pasa si aumentas o disminuyes la tasa de aprendizaje?

3. Utiliza al menos otros 2 métodos de optimización (existentes en Tensorflow), ajustalos y comparalos. ¿Cual de los métodos te gusta más y porque preferirías unos sobre otros?

4. ¿Que pasa si cambias el tamaño de los minibatches?
5. ¿Como harías si dejaste a medias un proceso de aprendizaje (en 10 epochs por ejemplo) y quisieras entrenar la red 10 epoch más, y mañana quisieras entrenarla otros 10 epoch más?

**Para contestar las preguntas, agrega cuantas celdas con comentarios y con códgo sean necesarias.** Aprovecha que las libretas de *Jupyter* te permite hacerte una especie de tutorial personalizado.

### Pregunta 1

In [22]:
archivo_modelo = "/tmp/rnn2_extra.ckpt"
saver = tf.train.Saver()

In [24]:
numero_epochs = 80
tamano_minibatch = 100
display_step = 10

# Muy importante la primera vez que se ejecuta inicializar todas las variables
init = tf.global_variables_initializer()

# La manera correcta de iniciar una sesión y realizar calculos
with tf.Session() as sess:
    sess.run(init)

    # Ciclos de entrenamiento
    for epoch in range(numero_epochs):

        #  Inicializa el costo promedio de todos los minibatches en 0
        avg_cost = 0.
        
        #  Calcula el número de minibatches que se pueden usar 
        total_batch = int(mnist.train.num_examples/tamano_minibatch)

        #  Por cada minibatch
        for i in range(total_batch):
            
            #  Utiliza un generador incluido en mnist que obtiene 
            #  tamano_minibatch ejemplos selecionados aleatoriamente del total
            batch_x, batch_y = mnist.train.next_batch(tamano_minibatch)
            
            #  Ejecuta la ops del paso_entrenamiento para aprender 
            #  y la del costo, con el fin de mostrar el aprendizaje
            _, c = sess.run([paso_entrenamiento, costo], feed_dict={x: batch_x, y: batch_y})
            
            #  Calcula el costo del minibatch y lo agrega al costo total
            avg_cost += c / total_batch
        
        # Muestra los resultados
        if epoch % display_step == 0:
            print (("Epoch: " + str(epoch)).ljust(20)
                   + ("Costo: " + str(avg_cost)))
    
    #  Guarda la sesión en el archivo rnn2_extra.cptk
    saver.save(sess, archivo_modelo)
    
    print("Se acabaron los epochs, saliendo de la sesión de tensorflow.")

Epoch: 0            Costo: 156.749482001
Epoch: 10           Costo: 1.82090597643
Epoch: 20           Costo: 0.277370307494
Epoch: 30           Costo: 0.209003690651
Epoch: 40           Costo: 0.209722215759
Epoch: 50           Costo: 0.154873590007
Epoch: 60           Costo: 0.098158695114
Epoch: 70           Costo: 0.0732445073968
Se acabaron los epochs, saliendo de la sesión de tensorflow.


In [25]:
prediction_correcta = tf.equal(tf.argmax(estimado, 1), tf.argmax(y, 1))

precision = tf.reduce_mean(tf.cast(prediction_correcta, "float"))

with tf.Session() as sess:
    sess.run(init)
    saver.restore(sess, archivo_modelo)
    porcentaje_acierto = sess.run(precision, feed_dict={x: mnist.test.images,
                                                        y: mnist.test.labels})
    print("Precisión: {}".format(porcentaje_acierto))

INFO:tensorflow:Restoring parameters from /tmp/rnn2_extra.ckpt
Precisión: 0.9664999842643738


El costo disminuyó y la presición aumento.

### Pregunta 2

In [27]:
alfa = 0.5
optimizador = tf.train.AdamOptimizer(learning_rate=alfa)
paso_entrenamiento = optimizador.minimize(costo)

archivo_modelo = "/tmp/rnn2_tasa_de_aprendizaje.ckpt"
saver = tf.train.Saver()

numero_epochs = 20
tamano_minibatch = 100
display_step = 1

# Muy importante la primera vez que se ejecuta inicializar todas las variables
init = tf.global_variables_initializer()

# La manera correcta de iniciar una sesión y realizar calculos
with tf.Session() as sess:
    sess.run(init)

    # Ciclos de entrenamiento
    for epoch in range(numero_epochs):

        #  Inicializa el costo promedio de todos los minibatches en 0
        avg_cost = 0.
        
        #  Calcula el número de minibatches que se pueden usar 
        total_batch = int(mnist.train.num_examples/tamano_minibatch)

        #  Por cada minibatch
        for i in range(total_batch):
            
            #  Utiliza un generador incluido en mnist que obtiene 
            #  tamano_minibatch ejemplos selecionados aleatoriamente del total
            batch_x, batch_y = mnist.train.next_batch(tamano_minibatch)
            
            #  Ejecuta la ops del paso_entrenamiento para aprender 
            #  y la del costo, con el fin de mostrar el aprendizaje
            _, c = sess.run([paso_entrenamiento, costo], feed_dict={x: batch_x, y: batch_y})
            
            #  Calcula el costo del minibatch y lo agrega al costo total
            avg_cost += c / total_batch
        
        # Muestra los resultados
        if epoch % display_step == 0:
            print (("Epoch: " + str(epoch)).ljust(20)
                   + ("Costo: " + str(avg_cost)))
    
    #  Guarda la sesión en el archivo rnn2.cptk
    saver.save(sess, archivo_modelo)
    
    print("Se acabaron los epochs, saliendo de la sesión de tensorflow.")

Epoch: 0            Costo: 232.374561063
Epoch: 1            Costo: 2.60299879378
Epoch: 2            Costo: 2.44583944928
Epoch: 3            Costo: 2.32425746007
Epoch: 4            Costo: 2.32719724395
Epoch: 5            Costo: 2.32947739471
Epoch: 6            Costo: 2.33917283838
Epoch: 7            Costo: 2.32492736903
Epoch: 8            Costo: 2.33055737712
Epoch: 9            Costo: 2.32730137478
Epoch: 10           Costo: 2.32851075909
Epoch: 11           Costo: 2.32831539761
Epoch: 12           Costo: 2.32626075961
Epoch: 13           Costo: 2.33121326013
Epoch: 14           Costo: 2.32892140388
Epoch: 15           Costo: 2.32748327515
Epoch: 16           Costo: 2.32798261556
Epoch: 17           Costo: 2.3309835538
Epoch: 18           Costo: 2.33034894987
Epoch: 19           Costo: 2.32825829463
Se acabaron los epochs, saliendo de la sesión de tensorflow.


In [28]:
with tf.Session() as sess:
    sess.run(init)
    saver.restore(sess, archivo_modelo)
    porcentaje_acierto = sess.run(precision, feed_dict={x: mnist.test.images,
                                                        y: mnist.test.labels})
    print("Precisión: {}".format(porcentaje_acierto))

INFO:tensorflow:Restoring parameters from /tmp/rnn2_tasa_de_aprendizaje.ckpt
Precisión: 0.11460000276565552


Si se aumenta el alfa, la presición es muy mala y el costo desde el segundo epoch varia muy poco arriba o abajo. 

In [29]:
alfa = 0.000001
optimizador = tf.train.AdamOptimizer(learning_rate=alfa)
paso_entrenamiento = optimizador.minimize(costo)

archivo_modelo = "/tmp/rnn2_tasa_de_aprendizaje.ckpt"
saver = tf.train.Saver()

numero_epochs = 20
tamano_minibatch = 100
display_step = 1

# Muy importante la primera vez que se ejecuta inicializar todas las variables
init = tf.global_variables_initializer()

# La manera correcta de iniciar una sesión y realizar calculos
with tf.Session() as sess:
    sess.run(init)

    # Ciclos de entrenamiento
    for epoch in range(numero_epochs):

        #  Inicializa el costo promedio de todos los minibatches en 0
        avg_cost = 0.
        
        #  Calcula el número de minibatches que se pueden usar 
        total_batch = int(mnist.train.num_examples/tamano_minibatch)

        #  Por cada minibatch
        for i in range(total_batch):
            
            #  Utiliza un generador incluido en mnist que obtiene 
            #  tamano_minibatch ejemplos selecionados aleatoriamente del total
            batch_x, batch_y = mnist.train.next_batch(tamano_minibatch)
            
            #  Ejecuta la ops del paso_entrenamiento para aprender 
            #  y la del costo, con el fin de mostrar el aprendizaje
            _, c = sess.run([paso_entrenamiento, costo], feed_dict={x: batch_x, y: batch_y})
            
            #  Calcula el costo del minibatch y lo agrega al costo total
            avg_cost += c / total_batch
        
        # Muestra los resultados
        if epoch % display_step == 0:
            print (("Epoch: " + str(epoch)).ljust(20)
                   + ("Costo: " + str(avg_cost)))
    
    #  Guarda la sesión en el archivo rnn2.cptk
    saver.save(sess, archivo_modelo)
    
    print("Se acabaron los epochs, saliendo de la sesión de tensorflow.")

Epoch: 0            Costo: 2535.40527588
Epoch: 1            Costo: 2411.14540017
Epoch: 2            Costo: 2311.7555471
Epoch: 3            Costo: 2198.69232511
Epoch: 4            Costo: 2098.41237971
Epoch: 5            Costo: 2013.07024791
Epoch: 6            Costo: 1920.88955012
Epoch: 7            Costo: 1834.25222567
Epoch: 8            Costo: 1762.03332586
Epoch: 9            Costo: 1687.44386164
Epoch: 10           Costo: 1623.55882058
Epoch: 11           Costo: 1560.05248047
Epoch: 12           Costo: 1505.29395708
Epoch: 13           Costo: 1458.1321824
Epoch: 14           Costo: 1412.20380748
Epoch: 15           Costo: 1363.48983975
Epoch: 16           Costo: 1324.84317594
Epoch: 17           Costo: 1289.53528809
Epoch: 18           Costo: 1253.56579102
Epoch: 19           Costo: 1218.42309548
Se acabaron los epochs, saliendo de la sesión de tensorflow.


In [30]:
with tf.Session() as sess:
    sess.run(init)
    saver.restore(sess, archivo_modelo)
    porcentaje_acierto = sess.run(precision, feed_dict={x: mnist.test.images,
                                                        y: mnist.test.labels})
    print("Precisión: {}".format(porcentaje_acierto))

INFO:tensorflow:Restoring parameters from /tmp/rnn2_tasa_de_aprendizaje.ckpt
Precisión: 0.1251000016927719


Si baja la tasa de aprendizaje, el costo es muchisimo para los primeros epoch, disminue pero es mucho. 
La presición es muy mala

### Pregunta 3

In [31]:
alfa = 0.001

optimizador = tf.train.AdagradOptimizer(learning_rate=alfa)
paso_entrenamiento = optimizador.minimize(costo)

archivo_modelo = "/tmp/rnn2_AdagradOptimizer.ckpt"
saver = tf.train.Saver()

numero_epochs = 30
tamano_minibatch = 100
display_step = 1

# Muy importante la primera vez que se ejecuta inicializar todas las variables
init = tf.global_variables_initializer()

# La manera correcta de iniciar una sesión y realizar calculos
with tf.Session() as sess:
    sess.run(init)

    # Ciclos de entrenamiento
    for epoch in range(numero_epochs):

        #  Inicializa el costo promedio de todos los minibatches en 0
        avg_cost = 0.
        
        #  Calcula el número de minibatches que se pueden usar 
        total_batch = int(mnist.train.num_examples/tamano_minibatch)

        #  Por cada minibatch
        for i in range(total_batch):
            
            #  Utiliza un generador incluido en mnist que obtiene 
            #  tamano_minibatch ejemplos selecionados aleatoriamente del total
            batch_x, batch_y = mnist.train.next_batch(tamano_minibatch)
            
            #  Ejecuta la ops del paso_entrenamiento para aprender 
            #  y la del costo, con el fin de mostrar el aprendizaje
            _, c = sess.run([paso_entrenamiento, costo], feed_dict={x: batch_x, y: batch_y})
            
            #  Calcula el costo del minibatch y lo agrega al costo total
            avg_cost += c / total_batch
        
        # Muestra los resultados
        if epoch % display_step == 0:
            print (("Epoch: " + str(epoch)).ljust(20)
                   + ("Costo: " + str(avg_cost)))
    
    #  Guarda la sesión en el archivo rnn2.cptk
    saver.save(sess, archivo_modelo)
    
    print("Se acabaron los epochs, saliendo de la sesión de tensorflow.")
    
    
prediction_correcta = tf.equal(tf.argmax(estimado, 1), tf.argmax(y, 1))

precision = tf.reduce_mean(tf.cast(prediction_correcta, "float"))

with tf.Session() as sess:
    sess.run(init)
    saver.restore(sess, archivo_modelo)
    porcentaje_acierto = sess.run(precision, feed_dict={x: mnist.test.images,
                                                        y: mnist.test.labels})
    print("Precisión: {}".format(porcentaje_acierto))

Epoch: 0            Costo: 547.486568825
Epoch: 1            Costo: 296.253083468
Epoch: 2            Costo: 233.831970145
Epoch: 3            Costo: 199.08785288
Epoch: 4            Costo: 180.591443787
Epoch: 5            Costo: 164.876214059
Epoch: 6            Costo: 152.917955641
Epoch: 7            Costo: 144.291925638
Epoch: 8            Costo: 137.652194956
Epoch: 9            Costo: 131.376366647
Epoch: 10           Costo: 125.793953261
Epoch: 11           Costo: 119.803635524
Epoch: 12           Costo: 117.670221544
Epoch: 13           Costo: 114.194863323
Epoch: 14           Costo: 109.795717454
Epoch: 15           Costo: 108.140873489
Epoch: 16           Costo: 104.609223598
Epoch: 17           Costo: 103.981510259
Epoch: 18           Costo: 99.7748545005
Epoch: 19           Costo: 98.3854717601
Epoch: 20           Costo: 95.9667021907
Epoch: 21           Costo: 95.8222146884
Epoch: 22           Costo: 92.0865458367
Epoch: 23           Costo: 92.2758466582
Epoch: 24        

In [32]:
optimizador = tf.train.AdadeltaOptimizer(learning_rate=alfa)
paso_entrenamiento = optimizador.minimize(costo)

archivo_modelo = "/tmp/rnn2_AdadeltaOptimizer.ckpt"
saver = tf.train.Saver()

numero_epochs = 30
tamano_minibatch = 100
display_step = 1

# Muy importante la primera vez que se ejecuta inicializar todas las variables
init = tf.global_variables_initializer()

# La manera correcta de iniciar una sesión y realizar calculos
with tf.Session() as sess:
    sess.run(init)

    # Ciclos de entrenamiento
    for epoch in range(numero_epochs):

        #  Inicializa el costo promedio de todos los minibatches en 0
        avg_cost = 0.
        
        #  Calcula el número de minibatches que se pueden usar 
        total_batch = int(mnist.train.num_examples/tamano_minibatch)

        #  Por cada minibatch
        for i in range(total_batch):
            
            #  Utiliza un generador incluido en mnist que obtiene 
            #  tamano_minibatch ejemplos selecionados aleatoriamente del total
            batch_x, batch_y = mnist.train.next_batch(tamano_minibatch)
            
            #  Ejecuta la ops del paso_entrenamiento para aprender 
            #  y la del costo, con el fin de mostrar el aprendizaje
            _, c = sess.run([paso_entrenamiento, costo], feed_dict={x: batch_x, y: batch_y})
            
            #  Calcula el costo del minibatch y lo agrega al costo total
            avg_cost += c / total_batch
        
        # Muestra los resultados
        if epoch % display_step == 0:
            print (("Epoch: " + str(epoch)).ljust(20)
                   + ("Costo: " + str(avg_cost)))
    
    #  Guarda la sesión en el archivo rnn2.cptk
    saver.save(sess, archivo_modelo)
    
    print("Se acabaron los epochs, saliendo de la sesión de tensorflow.")
    
    
prediction_correcta = tf.equal(tf.argmax(estimado, 1), tf.argmax(y, 1))

precision = tf.reduce_mean(tf.cast(prediction_correcta, "float"))

with tf.Session() as sess:
    sess.run(init)
    saver.restore(sess, archivo_modelo)
    porcentaje_acierto = sess.run(precision, feed_dict={x: mnist.test.images,
                                                        y: mnist.test.labels})
    print("Precisión: {}".format(porcentaje_acierto))

Epoch: 0            Costo: 1463.96313898
Epoch: 1            Costo: 1437.78431441
Epoch: 2            Costo: 1411.50755149
Epoch: 3            Costo: 1390.79849787
Epoch: 4            Costo: 1371.72693415
Epoch: 5            Costo: 1351.7441748
Epoch: 6            Costo: 1328.63283037
Epoch: 7            Costo: 1313.11934925
Epoch: 8            Costo: 1293.60580356
Epoch: 9            Costo: 1273.19881403
Epoch: 10           Costo: 1265.23230569
Epoch: 11           Costo: 1243.34088634
Epoch: 12           Costo: 1226.2460204
Epoch: 13           Costo: 1217.04532127
Epoch: 14           Costo: 1201.96119773
Epoch: 15           Costo: 1183.50299527
Epoch: 16           Costo: 1178.74619174
Epoch: 17           Costo: 1156.88432928
Epoch: 18           Costo: 1150.71885132
Epoch: 19           Costo: 1137.15163541
Epoch: 20           Costo: 1128.37658458
Epoch: 21           Costo: 1117.96210738
Epoch: 22           Costo: 1104.0675334
Epoch: 23           Costo: 1094.84133944
Epoch: 24          

Segun los resultados Adagrad con pocos epoch da mejores resultados que Adadelta

### Pregunta 4

Disminuyendo el tamaño de minibatches

In [33]:
optimizador = tf.train.AdamOptimizer(learning_rate=alfa)
paso_entrenamiento = optimizador.minimize(costo)

archivo_modelo = "/tmp/rnn2_AdadeltaOptimizer.ckpt"
saver = tf.train.Saver()

numero_epochs = 30
tamano_minibatch = 50
display_step = 1

# Muy importante la primera vez que se ejecuta inicializar todas las variables
init = tf.global_variables_initializer()

# La manera correcta de iniciar una sesión y realizar calculos
with tf.Session() as sess:
    sess.run(init)

    # Ciclos de entrenamiento
    for epoch in range(numero_epochs):

        #  Inicializa el costo promedio de todos los minibatches en 0
        avg_cost = 0.
        
        #  Calcula el número de minibatches que se pueden usar 
        total_batch = int(mnist.train.num_examples/tamano_minibatch)

        #  Por cada minibatch
        for i in range(total_batch):
            
            #  Utiliza un generador incluido en mnist que obtiene 
            #  tamano_minibatch ejemplos selecionados aleatoriamente del total
            batch_x, batch_y = mnist.train.next_batch(tamano_minibatch)
            
            #  Ejecuta la ops del paso_entrenamiento para aprender 
            #  y la del costo, con el fin de mostrar el aprendizaje
            _, c = sess.run([paso_entrenamiento, costo], feed_dict={x: batch_x, y: batch_y})
            
            #  Calcula el costo del minibatch y lo agrega al costo total
            avg_cost += c / total_batch
        
        # Muestra los resultados
        if epoch % display_step == 0:
            print (("Epoch: " + str(epoch)).ljust(20)
                   + ("Costo: " + str(avg_cost)))
    
    #  Guarda la sesión en el archivo rnn2.cptk
    saver.save(sess, archivo_modelo)
    
    print("Se acabaron los epochs, saliendo de la sesión de tensorflow.")
    
    
prediction_correcta = tf.equal(tf.argmax(estimado, 1), tf.argmax(y, 1))

precision = tf.reduce_mean(tf.cast(prediction_correcta, "float"))

with tf.Session() as sess:
    sess.run(init)
    saver.restore(sess, archivo_modelo)
    porcentaje_acierto = sess.run(precision, feed_dict={x: mnist.test.images,
                                                        y: mnist.test.labels})
    print("Precisión: {}".format(porcentaje_acierto))

Epoch: 0            Costo: 113.833812391
Epoch: 1            Costo: 29.1889469907
Epoch: 2            Costo: 17.400283014
Epoch: 3            Costo: 11.2334742687
Epoch: 4            Costo: 7.66382020025
Epoch: 5            Costo: 5.0905557183
Epoch: 6            Costo: 3.58053307244
Epoch: 7            Costo: 2.53056989474
Epoch: 8            Costo: 1.87741476537
Epoch: 9            Costo: 1.38222716176
Epoch: 10           Costo: 1.12338609176
Epoch: 11           Costo: 0.856491831094
Epoch: 12           Costo: 0.876900393605
Epoch: 13           Costo: 0.70028325735
Epoch: 14           Costo: 0.586119618986
Epoch: 15           Costo: 0.541635808712
Epoch: 16           Costo: 0.522072956351
Epoch: 17           Costo: 0.506551749766
Epoch: 18           Costo: 0.460936696606
Epoch: 19           Costo: 0.480104107422
Epoch: 20           Costo: 0.400414458197
Epoch: 21           Costo: 0.400223736826
Epoch: 22           Costo: 0.386412043954
Epoch: 23           Costo: 0.31041285006
Epoch: 

Aumentando el tamaño de los minibatches:

In [34]:
optimizador = tf.train.AdamOptimizer(learning_rate=alfa)
paso_entrenamiento = optimizador.minimize(costo)

archivo_modelo = "/tmp/rnn2_AdadeltaOptimizer.ckpt"
saver = tf.train.Saver()

numero_epochs = 30
tamano_minibatch = 200
display_step = 1

# Muy importante la primera vez que se ejecuta inicializar todas las variables
init = tf.global_variables_initializer()

# La manera correcta de iniciar una sesión y realizar calculos
with tf.Session() as sess:
    sess.run(init)

    # Ciclos de entrenamiento
    for epoch in range(numero_epochs):

        #  Inicializa el costo promedio de todos los minibatches en 0
        avg_cost = 0.
        
        #  Calcula el número de minibatches que se pueden usar 
        total_batch = int(mnist.train.num_examples/tamano_minibatch)

        #  Por cada minibatch
        for i in range(total_batch):
            
            #  Utiliza un generador incluido en mnist que obtiene 
            #  tamano_minibatch ejemplos selecionados aleatoriamente del total
            batch_x, batch_y = mnist.train.next_batch(tamano_minibatch)
            
            #  Ejecuta la ops del paso_entrenamiento para aprender 
            #  y la del costo, con el fin de mostrar el aprendizaje
            _, c = sess.run([paso_entrenamiento, costo], feed_dict={x: batch_x, y: batch_y})
            
            #  Calcula el costo del minibatch y lo agrega al costo total
            avg_cost += c / total_batch
        
        # Muestra los resultados
        if epoch % display_step == 0:
            print (("Epoch: " + str(epoch)).ljust(20)
                   + ("Costo: " + str(avg_cost)))
    
    #  Guarda la sesión en el archivo rnn2.cptk
    saver.save(sess, archivo_modelo)
    
    print("Se acabaron los epochs, saliendo de la sesión de tensorflow.")
    
    
prediction_correcta = tf.equal(tf.argmax(estimado, 1), tf.argmax(y, 1))

precision = tf.reduce_mean(tf.cast(prediction_correcta, "float"))

with tf.Session() as sess:
    sess.run(init)
    saver.restore(sess, archivo_modelo)
    porcentaje_acierto = sess.run(precision, feed_dict={x: mnist.test.images,
                                                        y: mnist.test.labels})
    print("Precisión: {}".format(porcentaje_acierto))

Epoch: 0            Costo: 327.436073428
Epoch: 1            Costo: 62.654628178
Epoch: 2            Costo: 41.0186844254
Epoch: 3            Costo: 29.6620252193
Epoch: 4            Costo: 23.2910805494
Epoch: 5            Costo: 18.1169719549
Epoch: 6            Costo: 14.1526223356
Epoch: 7            Costo: 12.5643103131
Epoch: 8            Costo: 9.81056209217
Epoch: 9            Costo: 7.9694752807
Epoch: 10           Costo: 6.86271150069
Epoch: 11           Costo: 5.25901451879
Epoch: 12           Costo: 4.58864948693
Epoch: 13           Costo: 3.58216240873
Epoch: 14           Costo: 3.01173014271
Epoch: 15           Costo: 2.41217140966
Epoch: 16           Costo: 1.95993366422
Epoch: 17           Costo: 1.72499992207
Epoch: 18           Costo: 1.26890168588
Epoch: 19           Costo: 1.04058510346
Epoch: 20           Costo: 0.793752232148
Epoch: 21           Costo: 0.634468115036
Epoch: 22           Costo: 0.560499553126
Epoch: 23           Costo: 0.468243935205
Epoch: 24     

In [ ]:
Al aumentar el tamaño de los minibatches la presición disminuyo un poco, y al disminuir la precisí